# Greenplum Database  Concepts Explained - Part 1
## 1. System Setup
### 1.1 Initialize database connection and setup global variable values

In [42]:
import os, re
from IPython.display import display_html

import pygments.lexers
from pygments import highlight
from pygments.formatters import HtmlFormatter

CONNECTION_STRING = os.getenv('AWSGPDBCONN')

cs = re.match('^postgresql:\/\/(\S+):(\S+)@(\S+):(\S+)\/(\S+)$', CONNECTION_STRING)

DB_USER   = cs.group(1)
DB_PWD    = cs.group(2)
DB_SERVER = cs.group(3)
DB_PORT   = cs.group(4)
DB_NAME   = cs.group(5)

%reload_ext sql

In [43]:
%sql $CONNECTION_STRING
%sql $DB_USER@$DB_NAME {"SELECT version();"}

1 rows affected.


version
"PostgreSQL 9.4.24 (Greenplum Database 6.10.1 build commit:efba04ce26ebb29b535a255a5e95d1f5ebfde94e) on x86_64-unknown-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit compiled on Aug 13 2020 02:55:59"


In [44]:
query = "SHOW gp_autostats_mode; \
ALTER DATABASE {} SET gp_autostats_mode TO 'NONE'; \
SHOW gp_autostats_mode;".format(DB_NAME)

%sql $DB_USER@$DB_NAME {''.join(query)}

1 rows affected.
Done.
1 rows affected.


gp_autostats_mode
on_no_stats


In [45]:
%%sql $DB_USER@$DB_NAME
SELECT version();

1 rows affected.


version
"PostgreSQL 9.4.24 (Greenplum Database 6.10.1 build commit:efba04ce26ebb29b535a255a5e95d1f5ebfde94e) on x86_64-unknown-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit compiled on Aug 13 2020 02:55:59"


## 2. The Amazon Customer Reviews Dataset

Over 130+ million customer reviews are available to researchers as part of this release. The data is available in TSV files in the `amazon-reviews-pds` S3 bucket in AWS US East Region. Each line in the data files corresponds to an individual review (tab delimited, with no quote and escape characters). Samples of the data are available in English and French; more details on the information in each column can be found [here](https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt).

If you use the AWS Command Line Interface, you can list data in the bucket with the `aws s3 ls` command:

`aws s3 ls s3://amazon-reviews-pds/tsv/`

To download data using the AWS Command Line Interface, you can use the `aws s3 cp` command. For instance, the following command will copy the file named `amazon_reviews_us_Camera_v1_00.tsv.gz` to your local directory:

`aws s3 cp s3://amazon-reviews-pds/tsv/<S3 File> <Local File>`

### 2.1 Copy source files from AWS S3
For our demo, we choose to download the available files into the `/home/gpadmin/data/` folder, using the `aws s3 cp <S3 File> <Local File>` command described before, as follows:

In [46]:
shfilecode = !pygmentize -f html -O full,style=friendly -l bash script/1-3-aws-s3-copy.sh
display_html('\n'.join(shfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2020 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 #!/bin/bash 

 export S3_DATA_DIR = /data1/tmp_s3_data

 if [ -d $S3_DATA_DIR ] ; then rm -rf $S3_DATA_DIR ; fi 

mkdir -p $S3_DATA_DIR 
 cd $S3_DATA_DIR 

ls -laF $S3_DATA_DIR 

aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz amazon_reviews_us_Wireless_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz amazon_reviews_us_Watches_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz amazon_reviews_us_Video_Games_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz amazon_reviews_us_Video_DVD_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz amazon_reviews_us_Video_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz amazon_reviews_us_Toys_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz amazon_reviews_us_Tools_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz amazon_reviews_us_Sports_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz amazon_reviews_us_Software_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz amazon_reviews_us_Shoes_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz amazon_reviews_us_Pet_Products_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz amazon_reviews_us_PC_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz amazon_reviews_us_Outdoors_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz amazon_reviews_us_Office_Products_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz amazon_reviews_us_Music_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz amazon_reviews_us_Major_Appliances_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz amazon_reviews_us_Luggage_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz amazon_reviews_us_Kitchen_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz amazon_reviews_us_Jewelry_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz amazon_reviews_us_Home_Improvement_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz amazon_reviews_us_Home_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz amazon_reviews_us_Grocery_v1_00.tsv.gz
aws s3 cp s3://amazon-re

In [47]:
!#script/1-3-aws-s3-copy.sh

## 3. Data Loading
### 3.1. Create the Schema (optional) and the Database Table to hold the dataset, as shown below:

In [48]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/2-1-create-db-schema-table.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2020 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP SCHEMA IF EXISTS demo CASCADE ; 

 CREATE SCHEMA demo ; 

 DROP TABLE IF EXISTS demo . amzn_reviews ; 

 CREATE TABLE demo . amzn_reviews ( 
 marketplace TEXT , 
 customer_id BIGINT , 
 review_id TEXT , 
 product_id TEXT , 
 product_parent BIGINT , 
 product_title TEXT , 
 product_category TEXT , 
 star_rating INTEGER , 
 helpful_votes INTEGER , 
 total_votes INTEGER , 
 vine TEXT , 
 verified_purchase TEXT , 
 review_headline TEXT , 
 review_body TEXT , 
 review_date DATE ) 
 DISTRIBUTED BY ( review_id );

In [49]:
query = !cat script/2-1-create-db-schema-table.sql
%sql $DB_USER@$DB_NAME {''.join(query)}

Done.
Done.
Done.
Done.


[]

In [50]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/2-2-count-table.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2020 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT COUNT ( * ) FROM demo . amzn_reviews ;

In [51]:
query = !cat script/2-2-count-table.sql
%sql $DB_USER@$DB_NAME {''.join(query)}

1 rows affected.


count
0


### 3.2. Load the Input Dataset using the gpload Utility
**gpload** is a data loading utility that acts as an interface to the Greenplum Database external table parallel loading feature. Using a load specification defined in a YAML formatted control file, gpload executes a load by invoking the Greenplum Database parallel file server (**gpfdist**), creating an external table definition based on the source data defined, and executing an *INSERT*, *UPDATE* or *MERGE* operation to load the source data into the target table in the database.

You can declare more than one file as input/source as long as the data is of the same format in all files specified. Additionally, if the files are compressed using **gzip** or **bzip2** (have a .gz or .bz2 file extension), the files will be uncompressed automatically (provided that gunzip or bunzip2 is in your path). You can also declare options such as the schema of the source data files, perform basic transformations, define custom delimiter and/or escape character(s), and many more. For the full list of available options, check the GPLoad Utility Reference available on [Pivotal Greenplum Database Documentation](https://gpdb.docs.pivotal.io/latest) (*Pivotal Greenplum Documentation > Utility Guide > Management Utility Reference > gpload*).

The operation, including any SQL commands specified in the SQL collection of the YAML control file, are performed as a single transaction to prevent inconsistent data when performing multiple, simultaneous load operations on a target table.

For our demo, we have prepared the *gpload_amzn_reviews.yaml* YAML control file, as shown here:

In [52]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l yaml script/3-2-gpload-amzn-reviews.yaml
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2020 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 VERSION : 1.0.0.1 
 GPLOAD : 
 INPUT : 
 - SOURCE : 
 FILE : 
 - /data1/tmp_s3_data/amazon_reviews_us*.tsv.gz 
 - FORMAT : text 
 - HEADER : true 
 - LOG_ERRORS : true 
 - MAX_LINE_LENGTH : 100000 
 - ERROR_LIMIT : 50000 
 OUTPUT : 
 - TABLE : demo.amzn_reviews 
 - MODE : insert 
 PRELOAD : 
 - TRUNCATE : true 
 - REUSE_TABLES : true

#### 3.2.1. Delete error log information for existing tables in the current database.

In [53]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/3-1-delete-error-log-info.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2020 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT gp_truncate_error_log ( 'demo.amzn_reviews' );

In [54]:
query = !cat script/3-1-delete-error-log-info.sql
%sql $DB_USER@$DB_NAME {''.join(query)}

1 rows affected.


gp_truncate_error_log
True


#### 3.2.2. Copy GPLoad YAML file across to the Database Server and execute

In [ ]:
cmd = "gpload -d {0} -h {1} -U {2} -f script/3-2-gpload-amzn-reviews.yaml -l ./gpload_amzn_reviews.log 2>&1".format(DB_NAME,DB_SERVER,DB_USER) 

#cmd = "gpload.py -d {0} -f script/3-2-gpload-amzn-reviews.yaml -l ./gpload_amzn_reviews.log 2>&1".format(DB_NAME) 
print(cmd)
!export GPHOME=/usr/local/greenplum-db-clients/ && export PATH=$GPHOME/bin:$PATH && /usr/local/greenplum-db-clients/greenplum_clients_path.sh  && /usr/local/greenplum-db-clients/greenplum_loaders_path.sh
!$cmd

gpload -d demo -h greenplum -U gpadmin -f script/3-2-gpload-amzn-reviews.yaml -l ./gpload_amzn_reviews.log 2>&1
2020-09-23 08:06:20|INFO|gpload session started 2020-09-23 08:06:20
2020-09-23 08:06:21|INFO|started gpfdist -p 8000 -P 9000 -f "/data1/tmp_s3_data/amazon_reviews_us*.tsv.gz" -t 30 -m 100000
2020-09-23 08:06:21|INFO|did not find an external table to reuse. creating ext_gpload_reusable_aa2543e4_fd73_11ea_8680_4284c3410404


### 3.3. Check gpload execution

Check **gpload** execution output (shown above and also available on *./gpload_amzn_reviews.log*), confirm successful loading of the data and/or identify any message which require ones attention and/or actions:

#### 3.3.1. Check the data has been properly loaded, by confirming row count shown above:

In [33]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/3-3-count-amzn-reviews.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2020 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT COUNT ( * ) FROM demo . amzn_reviews ;

In [36]:
query = !cat script/3-3-count-amzn-reviews.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

1 rows affected.


count
150955707


In [39]:
cmd = 'cat gpload_amzn_reviews.log\
    | grep -e '"'"'WARN|select'"'"'\
    | awk '"'"'BEGIN{FS="|";OFS=" "} {print $3}'"'"'\
    | awk '"'"'{print $1, "COUNT(*)", $3, $4, $5, $6, $7, $8}'"'"''
query = !$cmd
print(query)
%sql $DB_USER@$DB_NAME {''.join(query)}

["select COUNT(*) from gp_read_error_log('ext_gpload_reusable_9051c6c0_fce4_11ea_a476_4284c3410404') where cmdtime > to_timestamp('1600786919.22')"]
1 rows affected.


count
7622


### Continue to Part 2 of Greenplum Database Concepts Explained; [Basic Table Functions](AWS-GP-demo-2.ipynb).